# 🧬 Geometry-Complete Equivariant Diffusion
## De Novo Drug Design Training

**Selection Strategy**:
- Size stratification (small/medium/large pockets)
- Centroid clustering for diversity
- Pocket-level split (no leakage)

**Result**: 100 train pockets × 50 ligands = 5,000 pairs

## Cell 1: Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import torch
print(f'GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else "None!"}')

!pip install -q torch-geometric rdkit scipy numpy pyyaml tqdm gdown scikit-learn
print('✅ Setup complete')

## Cell 2: Clone Repository

In [ ]:
import os
REPO = '/content/drive/MyDrive/geom_diffusion'
if not os.path.exists(REPO):
    !git clone https://github.com/Nethrananda21/geom_diffusion.git {REPO}
%cd {REPO}
!git pull origin master

## Cell 3: Download Pre-processed Data

In [ ]:
import os
import gdown

DATA_DIR = '/content/data/crossdocked'
os.makedirs(DATA_DIR, exist_ok=True)

files = {
    'train_data.pkl': '1vJyxCIqCYwP3qj4THMofdSd1rZDEQpPG',
    'val_data.pkl': '1FpVNcdj0R5YOsaLQm6T4D5QOKZGI4Xc5'
}

for fname, fid in files.items():
    path = f'{DATA_DIR}/{fname}'
    if not os.path.exists(path):
        print(f'📥 Downloading {fname}...')
        gdown.download(id=fid, output=path, quiet=False)
    else:
        print(f'✅ {fname} exists')

## Cell 4: Intelligent Pocket Selection (Option B)

**Strategy**:
1. Filter pockets by size (≤250 atoms for T4)
2. Stratify: 40 small + 40 medium + 40 large = 120 pockets
3. Use centroid clustering for diversity within each size bin
4. Ensure ≥50 ligands per pocket
5. Family-level split (train 100, val 20)

In [ ]:
import pickle
import numpy as np
from collections import defaultdict
from sklearn.cluster import KMeans

np.random.seed(42)

# Load data
with open(f'{DATA_DIR}/train_data.pkl', 'rb') as f:
    full_data = pickle.load(f)
print(f'Loaded {len(full_data)} samples')

# Inspect first sample structure
sample = full_data[0]
print(f'\nSample keys: {sample.keys() if hasattr(sample, "keys") else type(sample)}')

# Group by pocket
pockets = defaultdict(list)
pocket_info = {}  # Store metadata

for i, sample in enumerate(full_data):
    # Extract pocket identifier
    if hasattr(sample, 'keys'):
        pocket_id = sample.get('pocket_id', sample.get('receptor', f'pocket_{i}'))
        pocket_coords = np.array(sample.get('pocket_coords', sample.get('pocket_pos', [])))
    else:
        # torch_geometric Data object
        pocket_id = getattr(sample, 'pocket_id', f'pocket_{i}')
        pocket_coords = sample.pocket_pos.numpy() if hasattr(sample, 'pocket_pos') else np.zeros((1,3))
    
    pockets[pocket_id].append(sample)
    
    # Store pocket info (first occurrence)
    if pocket_id not in pocket_info:
        pocket_info[pocket_id] = {
            'size': len(pocket_coords),
            'centroid': pocket_coords.mean(axis=0) if len(pocket_coords) > 0 else np.zeros(3)
        }

print(f'Found {len(pockets)} unique pockets')

# === FILTER 1: Size (Hard T4 constraint) ===
MAX_POCKET_ATOMS = 250
MIN_LIGANDS = 50

valid_pockets = [
    pid for pid, samples in pockets.items()
    if pocket_info[pid]['size'] <= MAX_POCKET_ATOMS 
    and len(samples) >= MIN_LIGANDS
]
print(f'After size/ligand filter: {len(valid_pockets)} pockets')

# === STRATIFY by pocket size ===
small = [p for p in valid_pockets if pocket_info[p]['size'] <= 150]
medium = [p for p in valid_pockets if 150 < pocket_info[p]['size'] <= 200]
large = [p for p in valid_pockets if 200 < pocket_info[p]['size'] <= 250]

print(f'\nSize distribution:')
print(f'  Small (≤150): {len(small)}')
print(f'  Medium (151-200): {len(medium)}')
print(f'  Large (201-250): {len(large)}')

# === SELECT with diversity (centroid clustering) ===
def select_diverse(pocket_list, n_select):
    """Select diverse pockets using centroid clustering"""
    if len(pocket_list) <= n_select:
        return pocket_list
    
    # Get centroids
    centroids = np.array([pocket_info[p]['centroid'] for p in pocket_list])
    
    # K-means clustering
    kmeans = KMeans(n_clusters=n_select, random_state=42, n_init=10)
    kmeans.fit(centroids)
    
    # Select one pocket per cluster (closest to centroid)
    selected = []
    for cluster_id in range(n_select):
        cluster_mask = kmeans.labels_ == cluster_id
        cluster_pockets = [pocket_list[i] for i in range(len(pocket_list)) if cluster_mask[i]]
        if cluster_pockets:
            selected.append(cluster_pockets[0])
    
    return selected

# Select 40 from each size bin (total 120)
n_per_bin = 40
selected_small = select_diverse(small, min(n_per_bin, len(small)))
selected_medium = select_diverse(medium, min(n_per_bin, len(medium)))
selected_large = select_diverse(large, min(n_per_bin, len(large)))

all_selected = selected_small + selected_medium + selected_large
print(f'\nSelected: {len(selected_small)} small + {len(selected_medium)} medium + {len(selected_large)} large = {len(all_selected)} pockets')

# === SPLIT: Train 100, Val 20 (pocket-level, no overlap) ===
np.random.shuffle(all_selected)
train_pockets = all_selected[:100]
val_pockets = all_selected[100:120]

print(f'\n✅ Train: {len(train_pockets)} pockets')
print(f'✅ Val: {len(val_pockets)} pockets')
print(f'✅ Overlap: {len(set(train_pockets) & set(val_pockets))} (must be 0!)')

# === CREATE FINAL DATASETS ===
LIGANDS_PER_POCKET = 50

train_samples = []
for pid in train_pockets:
    ligands = pockets[pid][:LIGANDS_PER_POCKET]
    train_samples.extend(ligands)

val_samples = []
for pid in val_pockets:
    ligands = pockets[pid][:LIGANDS_PER_POCKET]
    val_samples.extend(ligands)

print(f'\n📊 Final Dataset:')
print(f'   Train: {len(train_samples)} samples')
print(f'   Val: {len(val_samples)} samples')

# Save
with open(f'{DATA_DIR}/train_5k.pkl', 'wb') as f:
    pickle.dump(train_samples, f)
with open(f'{DATA_DIR}/val_1k.pkl', 'wb') as f:
    pickle.dump(val_samples, f)

print(f'\n💾 Saved: train_5k.pkl, val_1k.pkl')

## Cell 5: Update Config

In [ ]:
import yaml

%cd /content/drive/MyDrive/geom_diffusion

with open('configs/debug_t4.yaml', 'r') as f:
    cfg = yaml.safe_load(f)

cfg['data']['root'] = '/content/data'
cfg['data']['train_file'] = 'crossdocked/train_5k.pkl'
cfg['data']['val_file'] = 'crossdocked/val_1k.pkl'
cfg['training']['max_epochs'] = 50
cfg['hardware']['num_workers'] = 2

with open('configs/debug_t4.yaml', 'w') as f:
    yaml.dump(cfg, f)

print('✅ Config updated')

## Cell 6: Delete Cache

In [ ]:
import shutil
from pathlib import Path

for cache in ['/content/data/cache', 'data/cache']:
    if Path(cache).exists():
        shutil.rmtree(cache)
        print(f'🗑️ Deleted {cache}')

## Cell 7: Train 🚀

In [ ]:
%cd /content/drive/MyDrive/geom_diffusion
!python train.py --config configs/debug_t4.yaml --checkpoint_dir checkpoints

## Cell 8: Resume Training

In [ ]:
# Run Cells 1, 2, 5, 6 first, then:
# %cd /content/drive/MyDrive/geom_diffusion
# !python train.py --config configs/debug_t4.yaml --resume checkpoints/best_model.pt